<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Convert SoilGrids to COGs

_by Joe Hamman (CarbonPlan), October 27, 2020_

This notebook converts remote copies of SoilGrids rasters to cloud optimized
GeoTIFFs.

**Inputs:**

- urls to geotiffs

**Outputs:**

- Cloud optimized GeoTIFFs

**Notes:**

- No reprojection or processing of the data is done in this notebook.


In [ ]:
import io
import os
import pathlib

from rasterio.io import MemoryFile
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

In [ ]:
import rasterio as rio

In [ ]:
base_url = "https://files.isric.org/soilgrids/former/2017-03-10/data/"
fnames = [
    "ACDWRB_M_ss_250m_ll.tif",
    "AWCh1_M_sl1_250m_ll.tif",
    "AWCh1_M_sl2_250m_ll.tif",
    "AWCh1_M_sl3_250m_ll.tif",
    "AWCh1_M_sl4_250m_ll.tif",
    "AWCh1_M_sl5_250m_ll.tif",
    "AWCh1_M_sl6_250m_ll.tif",
    "AWCh1_M_sl7_250m_ll.tif",
    "AWCh2_M_sl1_250m_ll.tif",
    "AWCh2_M_sl2_250m_ll.tif",
    "AWCh2_M_sl3_250m_ll.tif",
    "AWCh2_M_sl4_250m_ll.tif",
    "AWCh2_M_sl5_250m_ll.tif",
    "AWCh2_M_sl6_250m_ll.tif",
    "AWCh2_M_sl7_250m_ll.tif",
    "AWCh3_M_sl1_250m_ll.tif",
    "AWCh3_M_sl2_250m_ll.tif",
    "AWCh3_M_sl3_250m_ll.tif",
    "AWCh3_M_sl4_250m_ll.tif",
    "AWCh3_M_sl5_250m_ll.tif",
    "AWCh3_M_sl6_250m_ll.tif",
    "AWCh3_M_sl7_250m_ll.tif",
    "AWCtS_M_sl1_250m_ll.tif",
    "AWCtS_M_sl2_250m_ll.tif",
    "AWCtS_M_sl3_250m_ll.tif",
    "AWCtS_M_sl4_250m_ll.tif",
    "AWCtS_M_sl5_250m_ll.tif",
    "AWCtS_M_sl6_250m_ll.tif",
    "AWCtS_M_sl7_250m_ll.tif",
    "BDRICM_M_250m_ll.tif",
    "BDRLOG_M_250m_ll.tif",
    "BDTICM_M_250m_ll.tif",
    "BLDFIE_M_sl1_250m_ll.tif",
    "BLDFIE_M_sl2_250m_ll.tif",
    "BLDFIE_M_sl3_250m_ll.tif",
    "BLDFIE_M_sl4_250m_ll.tif",
    "BLDFIE_M_sl5_250m_ll.tif",
    "BLDFIE_M_sl6_250m_ll.tif",
    "BLDFIE_M_sl7_250m_ll.tif",
    "CECSOL_M_sl1_250m_ll.tif",
    "CECSOL_M_sl2_250m_ll.tif",
    "CECSOL_M_sl3_250m_ll.tif",
    "CECSOL_M_sl4_250m_ll.tif",
    "CECSOL_M_sl5_250m_ll.tif",
    "CECSOL_M_sl6_250m_ll.tif",
    "CECSOL_M_sl7_250m_ll.tif",
    "CLYPPT_M_sl1_250m_ll.tif",
    "CLYPPT_M_sl2_250m_ll.tif",
    "CLYPPT_M_sl3_250m_ll.tif",
    "CLYPPT_M_sl4_250m_ll.tif",
    "CLYPPT_M_sl5_250m_ll.tif",
    "CLYPPT_M_sl6_250m_ll.tif",
    "CLYPPT_M_sl7_250m_ll.tif",
    "CRFVOL_M_sl1_250m_ll.tif",
    "CRFVOL_M_sl2_250m_ll.tif",
    "CRFVOL_M_sl3_250m_ll.tif",
    "CRFVOL_M_sl4_250m_ll.tif",
    "CRFVOL_M_sl5_250m_ll.tif",
    "CRFVOL_M_sl6_250m_ll.tif",
    "CRFVOL_M_sl7_250m_ll.tif",
    "HISTPR_250m_ll.tif",
    "OCDENS_M_sl1_250m_ll.tif",
    "OCDENS_M_sl2_250m_ll.tif",
    "OCDENS_M_sl3_250m_ll.tif",
    "OCDENS_M_sl4_250m_ll.tif",
    "OCDENS_M_sl5_250m_ll.tif",
    "OCDENS_M_sl6_250m_ll.tif",
    "OCDENS_M_sl7_250m_ll.tif",
    "OCSTHA_M_100cm_250m_ll.tif",
    "OCSTHA_M_200cm_250m_ll.tif",
    "OCSTHA_M_30cm_250m_ll.tif",
    "OCSTHA_M_sd1_250m_ll.tif",
    "OCSTHA_M_sd2_250m_ll.tif",
    "OCSTHA_M_sd3_250m_ll.tif",
    "OCSTHA_M_sd4_250m_ll.tif",
    "OCSTHA_M_sd5_250m_ll.tif",
    "OCSTHA_M_sd6_250m_ll.tif",
    "ORCDRC_M_sl1_250m_ll.tif",
    "ORCDRC_M_sl2_250m_ll.tif",
    "ORCDRC_M_sl3_250m_ll.tif",
    "ORCDRC_M_sl4_250m_ll.tif",
    "ORCDRC_M_sl5_250m_ll.tif",
    "ORCDRC_M_sl6_250m_ll.tif",
    "ORCDRC_M_sl7_250m_ll.tif",
    "PHIHOX_M_sl1_250m_ll.tif",
    "PHIHOX_M_sl2_250m_ll.tif",
    "PHIHOX_M_sl3_250m_ll.tif",
    "PHIHOX_M_sl4_250m_ll.tif",
    "PHIHOX_M_sl5_250m_ll.tif",
    "PHIHOX_M_sl6_250m_ll.tif",
    "PHIHOX_M_sl7_250m_ll.tif",
    "PHIKCL_M_sl1_250m_ll.tif",
    "PHIKCL_M_sl2_250m_ll.tif",
    "PHIKCL_M_sl3_250m_ll.tif",
    "PHIKCL_M_sl4_250m_ll.tif",
    "PHIKCL_M_sl5_250m_ll.tif",
    "PHIKCL_M_sl6_250m_ll.tif",
    "PHIKCL_M_sl7_250m_ll.tif",
    "SLGWRB_250m_ll.tif",
    "SLTPPT_M_sl1_250m_ll.tif",
    "SLTPPT_M_sl2_250m_ll.tif",
    "SLTPPT_M_sl3_250m_ll.tif",
    "SLTPPT_M_sl4_250m_ll.tif",
    "SLTPPT_M_sl5_250m_ll.tif",
    "SLTPPT_M_sl6_250m_ll.tif",
    "SLTPPT_M_sl7_250m_ll.tif",
    "SNDPPT_M_sl1_250m_ll.tif",
    "SNDPPT_M_sl2_250m_ll.tif",
    "SNDPPT_M_sl3_250m_ll.tif",
    "SNDPPT_M_sl4_250m_ll.tif",
    "SNDPPT_M_sl5_250m_ll.tif",
    "SNDPPT_M_sl6_250m_ll.tif",
    "SNDPPT_M_sl7_250m_ll.tif",
    "TAXNWRB_250m_ll.tif",
    "TAXNWRB_Acric.Ferralsols_250m_ll.tif",
    "TAXNWRB_Acric.Plinthosols_250m_ll.tif",
    "TAXNWRB_Albic.Arenosols_250m_ll.tif",
    "TAXNWRB_Albic.Luvisols_250m_ll.tif",
    "TAXNWRB_Alic.Nitisols_250m_ll.tif",
    "TAXNWRB_Aluandic.Andosols_250m_ll.tif",
    "TAXNWRB_Aric.Regosols_250m_ll.tif",
    "TAXNWRB_Calcaric.Regosols_250m_ll.tif",
    "TAXNWRB_Calcic.Chernozems_250m_ll.tif",
    "TAXNWRB_Calcic.Gleysols_250m_ll.tif",
    "TAXNWRB_Calcic.Gypsisols_250m_ll.tif",
    "TAXNWRB_Calcic.Histosols_250m_ll.tif",
    "TAXNWRB_Calcic.Kastanozems_250m_ll.tif",
    "TAXNWRB_Calcic.Luvisols_250m_ll.tif",
    "TAXNWRB_Calcic.Solonetz_250m_ll.tif",
    "TAXNWRB_Calcic.Vertisols_250m_ll.tif",
    "TAXNWRB_Cryic.Histosols_250m_ll.tif",
    "TAXNWRB_Cutanic.Alisols_250m_ll.tif",
    "TAXNWRB_Endogleyic.Cambisols_250m_ll.tif",
    "TAXNWRB_Endogleyic.Planosols_250m_ll.tif",
    "TAXNWRB_Ferralic.Arenosols_250m_ll.tif",
    "TAXNWRB_Ferralic.Cambisols_250m_ll.tif",
    "TAXNWRB_Fibric.Histosols_250m_ll.tif",
    "TAXNWRB_Gleyic.Luvisols_250m_ll.tif",
    "TAXNWRB_Gleyic.Podzols_250m_ll.tif",
    "TAXNWRB_Gleyic.Solonetz_250m_ll.tif",
    "TAXNWRB_Gypsic.Solonchaks_250m_ll.tif",
    "TAXNWRB_Haplic.Acrisols..Alumic._250m_ll.tif",
    "TAXNWRB_Haplic.Acrisols..Ferric._250m_ll.tif",
    "TAXNWRB_Haplic.Acrisols..Humic._250m_ll.tif",
    "TAXNWRB_Haplic.Acrisols_250m_ll.tif",
    "TAXNWRB_Haplic.Albeluvisols_250m_ll.tif",
    "TAXNWRB_Haplic.Alisols_250m_ll.tif",
    "TAXNWRB_Haplic.Andosols_250m_ll.tif",
    "TAXNWRB_Haplic.Arenosols..Calcaric._250m_ll.tif",
    "TAXNWRB_Haplic.Arenosols_250m_ll.tif",
    "TAXNWRB_Haplic.Calcisols..Sodic._250m_ll.tif",
    "TAXNWRB_Haplic.Calcisols_250m_ll.tif",
    "TAXNWRB_Haplic.Cambisols..Calcaric._250m_ll.tif",
    "TAXNWRB_Haplic.Cambisols..Chromic._250m_ll.tif",
    "TAXNWRB_Haplic.Cambisols..Dystric._250m_ll.tif",
    "TAXNWRB_Haplic.Cambisols..Eutric._250m_ll.tif",
    "TAXNWRB_Haplic.Cambisols..Humic._250m_ll.tif",
    "TAXNWRB_Haplic.Cambisols..Sodic._250m_ll.tif",
    "TAXNWRB_Haplic.Cambisols_250m_ll.tif",
    "TAXNWRB_Haplic.Chernozems_250m_ll.tif",
    "TAXNWRB_Haplic.Cryosols_250m_ll.tif",
    "TAXNWRB_Haplic.Ferralsols..Rhodic._250m_ll.tif",
    "TAXNWRB_Haplic.Ferralsols..Xanthic._250m_ll.tif",
    "TAXNWRB_Haplic.Ferralsols_250m_ll.tif",
    "TAXNWRB_Haplic.Fluvisols..Arenic._250m_ll.tif",
    "TAXNWRB_Haplic.Fluvisols..Calcaric._250m_ll.tif",
    "TAXNWRB_Haplic.Fluvisols..Dystric._250m_ll.tif",
    "TAXNWRB_Haplic.Fluvisols..Eutric._250m_ll.tif",
    "TAXNWRB_Haplic.Fluvisols_250m_ll.tif",
    "TAXNWRB_Haplic.Gleysols..Dystric._250m_ll.tif",
    "TAXNWRB_Haplic.Gleysols..Eutric._250m_ll.tif",
    "TAXNWRB_Haplic.Gleysols_250m_ll.tif",
    "TAXNWRB_Haplic.Gypsisols_250m_ll.tif",
    "TAXNWRB_Haplic.Kastanozems_250m_ll.tif",
    "TAXNWRB_Haplic.Leptosols..Eutric._250m_ll.tif",
    "TAXNWRB_Haplic.Leptosols_250m_ll.tif",
    "TAXNWRB_Haplic.Lixisols..Chromic._250m_ll.tif",
    "TAXNWRB_Haplic.Lixisols..Ferric._250m_ll.tif",
    "TAXNWRB_Haplic.Lixisols_250m_ll.tif",
    "TAXNWRB_Haplic.Luvisols..Chromic._250m_ll.tif",
    "TAXNWRB_Haplic.Luvisols..Ferric._250m_ll.tif",
    "TAXNWRB_Haplic.Luvisols_250m_ll.tif",
    "TAXNWRB_Haplic.Nitisols..Rhodic._250m_ll.tif",
    "TAXNWRB_Haplic.Phaeozems_250m_ll.tif",
    "TAXNWRB_Haplic.Planosols..Dystric._250m_ll.tif",
    "TAXNWRB_Haplic.Planosols..Eutric._250m_ll.tif",
    "TAXNWRB_Haplic.Podzols_250m_ll.tif",
    "TAXNWRB_Haplic.Regosols..Dystric._250m_ll.tif",
    "TAXNWRB_Haplic.Regosols..Eutric._250m_ll.tif",
    "TAXNWRB_Haplic.Regosols..Sodic._250m_ll.tif",
    "TAXNWRB_Haplic.Solonchaks..Sodic._250m_ll.tif",
    "TAXNWRB_Haplic.Solonchaks_250m_ll.tif",
    "TAXNWRB_Haplic.Solonetz_250m_ll.tif",
    "TAXNWRB_Haplic.Umbrisols_250m_ll.tif",
    "TAXNWRB_Haplic.Vertisols..Eutric._250m_ll.tif",
    "TAXNWRB_Haplic.Vertisols_250m_ll.tif",
    "TAXNWRB_Hemic.Histosols_250m_ll.tif",
    "TAXNWRB_Histic.Albeluvisols_250m_ll.tif",
    "TAXNWRB_Hypoluvic.Arenosols_250m_ll.tif",
    "TAXNWRB_Leptic.Cambisols_250m_ll.tif",
    "TAXNWRB_Leptic.Luvisols_250m_ll.tif",
    "TAXNWRB_Leptic.Phaeozems_250m_ll.tif",
    "TAXNWRB_Leptic.Regosols_250m_ll.tif",
    "TAXNWRB_Leptic.Umbrisols_250m_ll.tif",
    "TAXNWRB_Lithic.Leptosols_250m_ll.tif",
    "TAXNWRB_Lixic.Plinthosols_250m_ll.tif",
    "TAXNWRB_Luvic.Calcisols_250m_ll.tif",
    "TAXNWRB_Luvic.Chernozems_250m_ll.tif",
    "TAXNWRB_Luvic.Phaeozems_250m_ll.tif",
    "TAXNWRB_Luvic.Planosols_250m_ll.tif",
    "TAXNWRB_Luvic.Stagnosols_250m_ll.tif",
    "TAXNWRB_Mollic.Gleysols_250m_ll.tif",
    "TAXNWRB_Mollic.Leptosols_250m_ll.tif",
    "TAXNWRB_Mollic.Solonetz_250m_ll.tif",
    "TAXNWRB_Mollic.Vertisols_250m_ll.tif",
    "TAXNWRB_Petric.Calcisols_250m_ll.tif",
    "TAXNWRB_Petric.Durisols_250m_ll.tif",
    "TAXNWRB_Plinthic.Acrisols_250m_ll.tif",
    "TAXNWRB_Protic.Arenosols_250m_ll.tif",
    "TAXNWRB_Rendzic.Leptosols_250m_ll.tif",
    "TAXNWRB_Sapric.Histosols_250m_ll.tif",
    "TAXNWRB_Solodic.Planosols_250m_ll.tif",
    "TAXNWRB_Stagnic.Luvisols_250m_ll.tif",
    "TAXNWRB_Turbic.Cryosols_250m_ll.tif",
    "TAXNWRB_Umbric.Albeluvisols_250m_ll.tif",
    "TAXNWRB_Umbric.Ferralsols_250m_ll.tif",
    "TAXNWRB_Umbric.Gleysols_250m_ll.tif",
    "TAXNWRB_Vertic.Cambisols_250m_ll.tif",
    "TAXNWRB_Vertic.Luvisols_250m_ll.tif",
    "TAXNWRB_Vetic.Acrisols_250m_ll.tif",
    "TAXNWRB_Vitric.Andosols_250m_ll.tif",
    "TAXNWRB_Vitric.Cryosols_250m_ll.tif",
    "TAXOUSDA_250m_ll.tif",
    "TAXOUSDA_Albolls_250m_ll.tif",
    "TAXOUSDA_Aqualfs_250m_ll.tif",
    "TAXOUSDA_Aquands_250m_ll.tif",
    "TAXOUSDA_Aquents_250m_ll.tif",
    "TAXOUSDA_Aquepts_250m_ll.tif",
    "TAXOUSDA_Aquerts_250m_ll.tif",
    "TAXOUSDA_Aquods_250m_ll.tif",
    "TAXOUSDA_Aquolls_250m_ll.tif",
    "TAXOUSDA_Aquox_250m_ll.tif",
    "TAXOUSDA_Aquults_250m_ll.tif",
    "TAXOUSDA_Arents_250m_ll.tif",
    "TAXOUSDA_Argids_250m_ll.tif",
    "TAXOUSDA_Borolls_250m_ll.tif",
    "TAXOUSDA_Calcids_250m_ll.tif",
    "TAXOUSDA_Cambids_250m_ll.tif",
    "TAXOUSDA_Cryalfs_250m_ll.tif",
    "TAXOUSDA_Cryands_250m_ll.tif",
    "TAXOUSDA_Cryepts_250m_ll.tif",
    "TAXOUSDA_Cryids_250m_ll.tif",
    "TAXOUSDA_Cryods_250m_ll.tif",
    "TAXOUSDA_Cryolls_250m_ll.tif",
    "TAXOUSDA_Durids_250m_ll.tif",
    "TAXOUSDA_Fibrists_250m_ll.tif",
    "TAXOUSDA_Fluvents_250m_ll.tif",
    "TAXOUSDA_Folists_250m_ll.tif",
    "TAXOUSDA_Gelands_250m_ll.tif",
    "TAXOUSDA_Gelods_250m_ll.tif",
    "TAXOUSDA_Gypsids_250m_ll.tif",
    "TAXOUSDA_Hemists_250m_ll.tif",
    "TAXOUSDA_Histels_250m_ll.tif",
    "TAXOUSDA_Humods_250m_ll.tif",
    "TAXOUSDA_Humults_250m_ll.tif",
    "TAXOUSDA_Ochrepts_250m_ll.tif",
    "TAXOUSDA_Orthels_250m_ll.tif",
    "TAXOUSDA_Orthents_250m_ll.tif",
    "TAXOUSDA_Orthods_250m_ll.tif",
    "TAXOUSDA_Perox_250m_ll.tif",
    "TAXOUSDA_Psamments_250m_ll.tif",
    "TAXOUSDA_Rendolls_250m_ll.tif",
    "TAXOUSDA_Salids_250m_ll.tif",
    "TAXOUSDA_Saprists_250m_ll.tif",
    "TAXOUSDA_Torrands_250m_ll.tif",
    "TAXOUSDA_Torrerts_250m_ll.tif",
    "TAXOUSDA_Torrox_250m_ll.tif",
    "TAXOUSDA_Turbels_250m_ll.tif",
    "TAXOUSDA_Udalfs_250m_ll.tif",
    "TAXOUSDA_Udands_250m_ll.tif",
    "TAXOUSDA_Udepts_250m_ll.tif",
    "TAXOUSDA_Uderts_250m_ll.tif",
    "TAXOUSDA_Udolls_250m_ll.tif",
    "TAXOUSDA_Udox_250m_ll.tif",
    "TAXOUSDA_Udults_250m_ll.tif",
    "TAXOUSDA_Ustalfs_250m_ll.tif",
    "TAXOUSDA_Ustands_250m_ll.tif",
    "TAXOUSDA_Ustepts_250m_ll.tif",
    "TAXOUSDA_Usterts_250m_ll.tif",
    "TAXOUSDA_Ustolls_250m_ll.tif",
    "TAXOUSDA_Ustox_250m_ll.tif",
    "TAXOUSDA_Ustults_250m_ll.tif",
    "TAXOUSDA_Vitrands_250m_ll.tif",
    "TAXOUSDA_Xeralfs_250m_ll.tif",
    "TAXOUSDA_Xerands_250m_ll.tif",
    "TAXOUSDA_Xerepts_250m_ll.tif",
    "TAXOUSDA_Xererts_250m_ll.tif",
    "TAXOUSDA_Xerolls_250m_ll.tif",
    "TAXOUSDA_Xerults_250m_ll.tif",
    "TEXMHT_M_sl1_250m_ll.tif",
    "TEXMHT_M_sl2_250m_ll.tif",
    "TEXMHT_M_sl3_250m_ll.tif",
    "TEXMHT_M_sl4_250m_ll.tif",
    "TEXMHT_M_sl5_250m_ll.tif",
    "TEXMHT_M_sl6_250m_ll.tif",
    "TEXMHT_M_sl7_250m_ll.tif",
    "WWP_M_sl1_250m_ll.tif",
    "WWP_M_sl2_250m_ll.tif",
    "WWP_M_sl3_250m_ll.tif",
    "WWP_M_sl4_250m_ll.tif",
    "WWP_M_sl5_250m_ll.tif",
    "WWP_M_sl6_250m_ll.tif",
    "WWP_M_sl7_250m_ll.tif",
]

In [ ]:
riods = rio.open(base_url + fnames[0])
riods

In [ ]:
dst_profile = cog_profiles.get("deflate")

In [ ]:
def upload(src, target, bucket="carbonplan-data"):
    storage_client = storage.Client("carbonplan")
    bucket = storage_client.bucket(bucket)
    blob = bucket.blob(target)
    blob.upload_from_filename(src)

## Iterate over datasets


In [ ]:
# conus forest group
workdir = pathlib.Path("/Users/freeman/workdir/carbonplan_data_downloads/nftd/")
source = workdir / "conus_forestgroup" / "conus_forestgroup.img"
target = workdir / "conus_forestgroup" / "raster.tif"
cloud = "raw/nftd/conus_forestgroup/30m/raster.tif"

In [ ]:
# convert to cogs
[cog_translate(f["source"], f["target"], dst_profile) for f in files]

In [ ]:
# upload to cloud storage
[upload(f["target"], f["cloud"]) for f in files]